In [1]:
import os, sys

sys.path.append(os.environ["BUILD_WORKSPACE_DIRECTORY"])

In [6]:
from third_party.vit_pose.models.model import ViTPose

In [8]:
from third_party.vit_pose.configs.ViTPose_base_coco_256x192 import (
    model as model_cfg,
)
from third_party.vit_pose.configs.ViTPose_base_coco_256x192 import data_cfg
import torch

CKPT_PATH = "/home/tim/ws/experimental/vitpose-b-multi-coco.pth"

vit_pose = ViTPose(model_cfg)
img_size = data_cfg["image_size"]

ckpt = torch.load(CKPT_PATH)
if "state_dict" in ckpt:
    vit_pose.load_state_dict(ckpt["state_dict"])
else:
    vit_pose.load_state_dict(ckpt)
device = torch.device("cuda")
vit_pose.to(device).eval()
print("Model loaded")

Model loaded


In [9]:
vit_pose

ViTPose(
  (backbone): ViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=(2, 2))
    )
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_af

In [10]:
@torch.no_grad()
def inference(img, device) -> np.ndarray:

    # Prepare input data
    org_w, org_h = img.size
    img_tensor = (
        transforms.Compose(
            [
                transforms.Resize((img_size[1], img_size[0])),
                transforms.ToTensor(),
            ]
        )(img)
        .unsqueeze(0)
        .to(device)
    )

    # Feed to model
    heatmaps = vit_pose(img_tensor).detach().cpu().numpy()  # N, 17, h/4, w/4
    points, probs = keypoints_from_heatmaps(
        heatmaps=heatmaps,
        center=np.array([[org_w // 2, org_h // 2]]),
        scale=np.array([[org_w, org_h]]),
        unbiased=True,
        use_udp=True,
    )
    return points, probs

NameError: name 'np' is not defined

In [13]:
image_addres = "https://github.com/jaehyunnn/ViTPose_pytorch/blob/main/examples/img1.jpg?raw=true"

# https://stackoverflow.com/questions/21061814/how-can-i-read-an-image-from-an-internet-url-in-python-cv2-scikit-image-and-mah
from urllib.request import urlopen
import cv2
import numpy as np


def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)

    # return the image
    return image


img = url_to_image(image_addres)

In [15]:
img.shape

(798, 720, 3)

In [19]:
transposed = img.transpose(2, 0, 1)
print(transposed.shape)
img_torch = torch.tensor(transposed)

(3, 798, 720)


In [22]:
img_torch = img_torch.unsqueeze(0)

In [24]:
vit_pose = vit_pose.to("cuda")

In [29]:
from torchvision import transforms

img_tensor = transforms.Compose(
    [transforms.Resize((img_size[1], img_size[0]))]
)(img_torch).cuda()


vit_pose(img_torch.cuda())

RuntimeError: Input type (unsigned char) and bias type (float) should be the same